In [ ]:
#判断是否安装了cuda
import torch
print(torch.cuda.is_available())  #返回True则说明已经安装了cuda
#判断是否安装了cuDNN
from torch.backends import  cudnn
print(cudnn.is_available())  #返回True则说明已经安装了cuDNN

In [1]:
'''
This is the main code of the 
ECAPATDNN project, to define the parameters and build the construction
'''

import argparse, glob, os, torch, warnings, time
from tools import *
from dataLoader import train_loader
# from DatasetLoader import train_dataset_loader
from ECAPAModel import ECAPAModel

In [2]:
parser = argparse.ArgumentParser(description = "ECAPA_trainer")
## Training Settings
parser.add_argument('--num_frames', type=int,   default=200,     help='Duration of the input segments, eg: 200 for 2 second')
parser.add_argument('--max_frames', type=int,   default=200,     help='Duration of the input segments, eg: 200 for 2 second')
parser.add_argument('--max_epoch',  type=int,   default=100,      help='Maximum number of epochs')
parser.add_argument('--batch_size', type=int,   default=256,     help='Batch size')
parser.add_argument('--n_cpu',      type=int,   default=16,      help='Number of loader threads')
# parser.add_argument('--augment',    type=bool,  default=True,   help='Augment input')
parser.add_argument('--test_step',  type=int,   default=1,       help='Test and save every [test_step] epochs')
parser.add_argument('--lr',         type=float, default=0.001,   help='Learning rate')
parser.add_argument("--lr_decay",   type=float, default=0.97,    help='Learning rate decay every [test_step] epochs')

## Training and evaluation path/lists, save path
parser.add_argument('--train_list',     type=str,   default="/root/autodl-tmp/Vox2/train_list.txt",        help='Train list')
parser.add_argument('--train_path',     type=str,   default="/root/autodl-tmp/Vox2/train",                 help='Absolute path to the train set')
parser.add_argument('--eval_list',      type=str,   default="/root/autodl-tmp/Vox1/test_list.txt",         help='Evaluation list')
parser.add_argument('--eval_path',      type=str,   default="/root/autodl-tmp/Vox1/test",                  help='Absolute path to the test set')
parser.add_argument('--musan_path',     type=str,   default="/root/autodl-tmp/musan",                      help='Absolute path to the MUSAN set')
parser.add_argument('--rir_path',       type=str,   default="/root/autodl-tmp/RIRS_NOISES/simulated_rirs", help='Absolute path to the RIR set')
parser.add_argument('--save_path',      type=str,   default="exps/Dual_TDNN_SDGF1536",                             help='Path to save the score.txt and models')
parser.add_argument('--initial_model',  type=str,   default="exps/Dual_TDNN_SDGF1536/model/model_0075.model",      help='Path of the initial_model') 

## Training and evaluation path/lists, save path
# parser.add_argument('--train_list', type=str,   default="D:/Dataset/Vox2/train_list.txt",          help='The path of the training list, https://www.robots.ox.ac.uk/~vgg/data/voxceleb/meta/train_list.txt')
# parser.add_argument('--train_path', type=str,   default="D:/Dataset/Vox2/train",                   help='The path of the training data, eg:"/data08/VoxCeleb2/train/wav" in my case')
# parser.add_argument('--eval_list',  type=str,   default="D:/Dataset/Vox1/test_list.txt",           help='The path of the evaluation list, veri_test2.txt comes from https://www.robots.ox.ac.uk/~vgg/data/voxceleb/meta/veri_test2.txt')
# parser.add_argument('--eval_path',  type=str,   default="D:/Dataset/Vox1/test",                    help='The path of the evaluation data, eg:"/data08/VoxCeleb1/test/wav" in my case')
# parser.add_argument('--musan_path', type=str,   default="D:/Dataset/musan",                        help='The path to the MUSAN set, eg:"/data08/Others/musan_split" in my case')
# parser.add_argument('--rir_path',   type=str,   default="D:/Dataset/RIRS_NOISES/simulated_rirs",   help='The path to the RIR set, eg:"/data08/Others/RIRS_NOISES/simulated_rirs" in my case');
# parser.add_argument('--save_path',  type=str,   default="exps/exp_dam128",                           help='Path to save the score.txt and models')
# parser.add_argument('--initial_model',  type=str,   default="exps/exp_dam128/model/model_0026.model",    help='Path of the initial_model') # exps/pretrain.model

## Model and Loss settings
parser.add_argument('--C',          type=int,   default=1536,   help='Channel size for the speaker encoder')
parser.add_argument('--mixedprec',  type=bool,  default=False,  help='Only do evaluation')
parser.add_argument('--m',          type=float, default=0.2,    help='Loss margin in AAM softmax')
parser.add_argument('--s',          type=float, default=30,     help='Loss scale in AAM softmax')
# parser.add_argument('--k',       type=int,   default=17,     help='subcenter nums in DAM softmax')
parser.add_argument('--n_class',    type=int,   default=5994,   help='Number of speakers')

## Command
# parser.add_argument('--eval',       dest='eval', action='store_false', help='Only do evaluation')
parser.add_argument('--eval',       type=bool,  default=False,  help='Only do evaluation')


## Initialization
warnings.simplefilter("ignore")
torch.multiprocessing.set_sharing_strategy('file_system')
args = parser.parse_args([])
args = init_args(args)

In [3]:
args.eval

False

In [4]:
## Define the data loader
trainloader = train_loader(**vars(args))
trainLoader = torch.utils.data.DataLoader(trainloader, batch_size = args.batch_size, shuffle = True, \
                                          pin_memory=True, num_workers = args.n_cpu, drop_last = True, prefetch_factor = 10)

# Search for the exist models
# modelfiles = glob.glob('%s/model_0*.model'%args.model_save_path)
# modelfiles.sort()

# #Only do evaluation, the initial_model is necessary
# if args.eval == True:
# 	s = ECAPAModel(**vars(args))
# 	print("Model %s loaded from previous state!"%args.initial_model)
# 	s.load_parameters(args.initial_model)
# 	EER, minDCF = s.eval_network(eval_list = args.eval_list, eval_path = args.eval_path)
# 	print("EER %2.2f%%, minDCF %.4f%%"%(EER, minDCF))
# 	quit()

# If initial_model is exist, system will train from the initial_model
# if args.initial_model != "":
# 	print("Model %s loaded from previous state!"%args.initial_model)
# 	s = ECAPAModel(**vars(args))
# 	s.load_parameters(args.initial_model)
# 	epoch = 1

# ## Otherwise, system will try to start from the saved model&epoch
# elif len(modelfiles) >= 1:
# 	print("Model %s loaded from previous state!"%modelfiles[-1])
# 	epoch = int(os.path.splitext(os.path.basename(modelfiles[-1]))[0][6:]) + 1
# 	s = ECAPAModel(**vars(args))
# 	s.load_parameters(modelfiles[-1])
## Otherwise, system will train from scratch
# else:

# 	epoch = 1
# 	s = ECAPAModel(**vars(args))
    
    
epoch = 1
s = ECAPAModel(**vars(args))

EERs = []
score_file = open(args.score_save_path, "a+")

03-13 15:38:13 Model para number = 19.91


In [ ]:
while(1):
	## Training for one epoch
	loss, lr, acc = s.train_network(epoch = epoch, loader = trainLoader)

	## Evaluation every [test_step] epochs
	if epoch % args.test_step == 0:
		s.save_parameters(args.model_save_path + "/model_%04d.model"%epoch)
		EERs.append(s.eval_network(eval_list = args.eval_list, eval_path = args.eval_path)[0])
		print(time.strftime("%Y-%m-%d %H:%M:%S"), "%d epoch, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%"%(epoch, acc, EERs[-1], min(EERs)))
		score_file.write("%d epoch, LR %f, LOSS %f, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%\n"%(epoch, lr, loss, acc, EERs[-1], min(EERs)))
		score_file.flush()

	if epoch >= args.max_epoch:
		quit()

	epoch += 1

In [ ]:
a

In [ ]:
while(1):
	## Training for one epoch
	loss, lr, acc = s.train_network(epoch = epoch, loader = trainLoader)

	## Evaluation every [test_step] epochs
	if epoch % args.test_step == 0:
		s.save_parameters(args.model_save_path + "/model_%04d.model"%epoch)
		EERs.append(s.eval_network(eval_list = args.eval_list, eval_path = args.eval_path)[0])
		print(time.strftime("%Y-%m-%d %H:%M:%S"), "%d epoch, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%"%(epoch, acc, EERs[-1], min(EERs)))
		score_file.write("%d epoch, LR %f, LOSS %f, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%\n"%(epoch, lr, loss, acc, EERs[-1], min(EERs)))
		score_file.flush()

	if epoch >= args.max_epoch:
		quit()

	epoch += 1

100%|██████████| 4366/4366 [00:44<00:00, 97.74it/s] 


2023-02-14 21:22:59 1 epoch, ACC 3.57%, EER 6.73%, bestEER 6.73%


In [ ]:
while(1):
	## Training for one epoch
	loss, lr, acc = s.train_network(epoch = epoch, loader = trainLoader)

	## Evaluation every [test_step] epochs
	if epoch % args.test_step == 0:
		s.save_parameters(args.model_save_path + "/model_%04d.model"%epoch)
		EERs.append(s.eval_network(eval_list = args.eval_list, eval_path = args.eval_path)[0])
		print(time.strftime("%Y-%m-%d %H:%M:%S"), "%d epoch, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%"%(epoch, acc, EERs[-1], min(EERs)))
		score_file.write("%d epoch, LR %f, LOSS %f, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%\n"%(epoch, lr, loss, acc, EERs[-1], min(EERs)))
		score_file.flush()

	if epoch >= args.max_epoch:
		quit()

	epoch += 1

100%|██████████| 4366/4366 [01:12<00:00, 60.33it/s]


2023-02-14 00:14:56 1 epoch, ACC 3.59%, EER 7.63%, bestEER 7.63%


100%|██████████| 4366/4366 [01:21<00:00, 53.29it/s]


2023-02-14 01:41:59 2 epoch, ACC 12.15%, EER 5.65%, bestEER 5.65%


100%|██████████| 4366/4366 [01:19<00:00, 55.23it/s]


...

2023-02-14 14:12:26 11 epoch, ACC 32.25%, EER 3.12%, bestEER 3.09%


100%|██████████| 4366/4366 [00:59<00:00, 73.82it/s]


2023-02-14 15:35:34 12 epoch, ACC 33.16%, EER 2.94%, bestEER 2.94%


100%|██████████| 4366/4366 [01:09<00:00, 63.00it/s]


2023-02-14 16:58:43 13 epoch, ACC 33.93%, EER 3.08%, bestEER 2.94%


In [ ]:
while(1):
	## Training for one epoch
	loss, lr, acc = s.train_network(epoch = epoch, loader = trainLoader)

	## Evaluation every [test_step] epochs
	if epoch % args.test_step == 0:
		s.save_parameters(args.model_save_path + "/model_%04d.model"%epoch)
		EERs.append(s.eval_network(eval_list = args.eval_list, eval_path = args.eval_path)[0])
		print(time.strftime("%Y-%m-%d %H:%M:%S"), "%d epoch, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%"%(epoch, acc, EERs[-1], min(EERs)))
		score_file.write("%d epoch, LR %f, LOSS %f, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%\n"%(epoch, lr, loss, acc, EERs[-1], min(EERs)))
		score_file.flush()

	if epoch >= args.max_epoch:
		quit()

	epoch += 1

100%|██████████| 4366/4366 [01:12<00:00, 59.85it/s]


2023-02-13 13:24:42 1 epoch, ACC 4.03%, EER 7.38%, bestEER 7.38%


100%|██████████| 4366/4366 [01:03<00:00, 68.61it/s]


2023-02-13 14:39:56 2 epoch, ACC 12.12%, EER 5.73%, bestEER 5.73%


100%|██████████| 4366/4366 [01:06<00:00, 65.84it/s]


2023-02-13 15:57:46 3 epoch, ACC 16.80%, EER 5.09%, bestEER 5.09%


In [ ]:
while(1):
	## Training for one epoch
	loss, lr, acc = s.train_network(epoch = epoch, loader = trainLoader)

	## Evaluation every [test_step] epochs
	if epoch % args.test_step == 0:
		s.save_parameters(args.model_save_path + "/model_%04d.model"%epoch)
		EERs.append(s.eval_network(eval_list = args.eval_list, eval_path = args.eval_path)[0])
		print(time.strftime("%Y-%m-%d %H:%M:%S"), "%d epoch, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%"%(epoch, acc, EERs[-1], min(EERs)))
		score_file.write("%d epoch, LR %f, LOSS %f, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%\n"%(epoch, lr, loss, acc, EERs[-1], min(EERs)))
		score_file.flush()

	if epoch >= args.max_epoch:
		quit()

	epoch += 1

100%|██████████| 4366/4366 [01:53<00:00, 38.62it/s]


2023-02-12 19:04:15 1 epoch, ACC 3.54%, EER 8.23%, bestEER 8.23%


100%|██████████| 4366/4366 [01:44<00:00, 41.78it/s]


2023-02-12 20:29:36 2 epoch, ACC 11.18%, EER 6.18%, bestEER 6.18%


100%|██████████| 4366/4366 [01:47<00:00, 40.51it/s]


...

2023-02-13 06:00:43 8 epoch, ACC 24.73%, EER 3.85%, bestEER 3.85%


100%|██████████| 4366/4366 [01:48<00:00, 40.29it/s]


2023-02-13 07:32:48 9 epoch, ACC 25.66%, EER 3.98%, bestEER 3.85%


100%|██████████| 4366/4366 [01:45<00:00, 41.51it/s]


2023-02-13 09:10:37 10 epoch, ACC 26.49%, EER 3.89%, bestEER 3.85%


In [ ]:
1

In [4]:
while(1):
	## Training for one epoch
	loss, lr, acc = s.train_network(epoch = epoch, loader = trainLoader)

	## Evaluation every [test_step] epochs
	if epoch % args.test_step == 0:
		s.save_parameters(args.model_save_path + "/model_%04d.model"%epoch)
		EERs.append(s.eval_network(eval_list = args.eval_list, eval_path = args.eval_path)[0])
		print(time.strftime("%Y-%m-%d %H:%M:%S"), "%d epoch, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%"%(epoch, acc, EERs[-1], min(EERs)))
		score_file.write("%d epoch, LR %f, LOSS %f, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%\n"%(epoch, lr, loss, acc, EERs[-1], min(EERs)))
		score_file.flush()

	if epoch >= args.max_epoch:
		quit()

	epoch += 1

OutOfMemoryError: CUDA out of memory. Tried to allocate 808.00 MiB (GPU 0; 23.69 GiB total capacity; 22.00 GiB already allocated; 522.56 MiB free; 22.86 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
EER, minDCF = s.eval_network(args.eval_list, args.eval_path)

  9%|▉         | 384/4366 [00:08<01:17, 51.60it/s]

In [6]:
EER

37.30584741891275

In [7]:
minDCF

0.9818410232983107

In [4]:
EER, minDCF = s.eval_network(args.eval_list, args.eval_path)

100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:34<00:00, 46.00it/s]


In [5]:
print(EER, minDCF)

1.0391859709893916 0.083946741718986


In [8]:
EER, minDCF = s.eval_network(args.eval_list, args.eval_path)
print(EER, minDCF)

100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:19<00:00, 54.91it/s]


1.3260445983979217 0.09368911019701155


In [ ]:
# Train DAM
while(1):
	## Training for one epoch
	loss, lr, acc = s.train_network(epoch = epoch, loader = trainLoader)

	## Evaluation every [test_step] epochs
	if epoch % args.test_step == 0:
		s.save_parameters(args.model_save_path + "/model_%04d.model"%epoch)
		EERs.append(s.eval_network(eval_list = args.eval_list, eval_path = args.eval_path)[0])
		print(time.strftime("%Y-%m-%d %H:%M:%S"), "%d epoch, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%"%(epoch, acc, EERs[-1], min(EERs)))
		score_file.write("%d epoch, LR %f, LOSS %f, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%\n"%(epoch, lr, loss, acc, EERs[-1], min(EERs)))
		score_file.flush()

	if epoch >= args.max_epoch:
		quit()

	epoch += 1

100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:38<00:00, 44.16it/s]


2023-01-08 15:44:46 1 epoch, ACC 0.85%, EER 24.94%, bestEER 24.94%


100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:34<00:00, 46.17it/s]


2023-01-08 20:12:55 2 epoch, ACC 4.30%, EER 22.81%, bestEER 22.81%


100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:34<00:00, 46.07it/s]


2023-01-09 00:42:32 3 epoch, ACC 7.17%, EER 24.08%, bestEER 22.81%


100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:34<00:00, 46.28it/s]


2023-01-09 05:10:59 4 epoch, ACC 9.42%, EER 23.64%, bestEER 22.81%


100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:35<00:00, 45.73it/s]


2023-01-09 09:38:55 5 epoch, ACC 11.29%, EER 23.42%, bestEER 22.81%


In [ ]:
# Train AAM
while(1):
	## Training for one epoch
	loss, lr, acc = s.train_network(epoch = epoch, loader = trainLoader)

	## Evaluation every [test_step] epochs
	if epoch % args.test_step == 0:
		s.save_parameters(args.model_save_path + "/model_%04d.model"%epoch)
		EERs.append(s.eval_network(eval_list = args.eval_list, eval_path = args.eval_path)[0])
		print(time.strftime("%Y-%m-%d %H:%M:%S"), "%d epoch, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%"%(epoch, acc, EERs[-1], min(EERs)))
		score_file.write("%d epoch, LR %f, LOSS %f, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%\n"%(epoch, lr, loss, acc, EERs[-1], min(EERs)))
		score_file.flush()

	if epoch >= args.max_epoch:
		quit()

	epoch += 1

100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:32<00:00, 46.90it/s]


2023-01-09 13:10:18 1 epoch, ACC 3.16%, EER 6.53%, bestEER 6.53%


100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:33<00:00, 46.53it/s]


2023-01-09 16:31:54 2 epoch, ACC 14.21%, EER 4.43%, bestEER 4.43%


In [ ]:
# Train subcenter_AAM
while(1):
	## Training for one epoch
	loss, lr, acc = s.train_network(epoch = epoch, loader = trainLoader)

	## Evaluation every [test_step] epochs
	if epoch % args.test_step == 0:
		s.save_parameters(args.model_save_path + "/model_%04d.model"%epoch)
		EERs.append(s.eval_network(eval_list = args.eval_list, eval_path = args.eval_path)[0])
		print(time.strftime("%Y-%m-%d %H:%M:%S"), "%d epoch, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%"%(epoch, acc, EERs[-1], min(EERs)))
		score_file.write("%d epoch, LR %f, LOSS %f, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%\n"%(epoch, lr, loss, acc, EERs[-1], min(EERs)))
		score_file.flush()

	if epoch >= args.max_epoch:
		quit()

	epoch += 1

100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:35<00:00, 45.59it/s]


2023-01-09 20:23:11 1 epoch, ACC 0.04%, EER 7.11%, bestEER 7.11%


100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:34<00:00, 45.90it/s]


2023-01-09 23:46:48 2 epoch, ACC 0.34%, EER 5.34%, bestEER 5.34%


100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:35<00:00, 45.77it/s]


2023-01-10 03:16:56 3 epoch, ACC 0.86%, EER 4.17%, bestEER 4.17%


100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:35<00:00, 45.79it/s]


2023-01-10 06:40:07 4 epoch, ACC 1.46%, EER 3.92%, bestEER 3.92%


100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:35<00:00, 45.61it/s]


2023-01-10 10:03:24 5 epoch, ACC 2.15%, EER 3.54%, bestEER 3.54%


In [4]:
# Train subcenter_AAM2
while(1):
	## Training for one epoch
	loss, lr, acc = s.train_network(epoch = epoch, loader = trainLoader)

	## Evaluation every [test_step] epochs
	if epoch % args.test_step == 0:
		s.save_parameters(args.model_save_path + "/model_%04d.model"%epoch)
		EERs.append(s.eval_network(eval_list = args.eval_list, eval_path = args.eval_path)[0])
		print(time.strftime("%Y-%m-%d %H:%M:%S"), "%d epoch, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%"%(epoch, acc, EERs[-1], min(EERs)))
		score_file.write("%d epoch, LR %f, LOSS %f, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%\n"%(epoch, lr, loss, acc, EERs[-1], min(EERs)))
		score_file.flush()

	if epoch >= args.max_epoch:
		quit()

	epoch += 1

100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:38<00:00, 44.34it/s]


2023-01-10 14:02:28 1 epoch, ACC 3.11%, EER 7.40%, bestEER 7.40%


100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:34<00:00, 46.15it/s]


2023-01-10 17:28:57 2 epoch, ACC 13.84%, EER 4.62%, bestEER 4.62%


100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:34<00:00, 46.10it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:36<00:00, 45.31it/s]


2023-01-11 17:32:43 9 epoch, ACC 43.65%, EER 2.08%, bestEER 2.08%


100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:36<00:00, 45.07it/s]


2023-01-11 21:01:58 10 epoch, ACC 45.47%, EER 2.03%, bestEER 2.03%


KeyboardInterrupt: 

In [ ]:
# Train subcenter_DAM
while(1):
	## Training for one epoch
	loss, lr, acc = s.train_network(epoch = epoch, loader = trainLoader)

	## Evaluation every [test_step] epochs
	if epoch % args.test_step == 0:
		s.save_parameters(args.model_save_path + "/model_%04d.model"%epoch)
		EERs.append(s.eval_network(eval_list = args.eval_list, eval_path = args.eval_path)[0])
		print(time.strftime("%Y-%m-%d %H:%M:%S"), "%d epoch, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%"%(epoch, acc, EERs[-1], min(EERs)))
		score_file.write("%d epoch, LR %f, LOSS %f, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%\n"%(epoch, lr, loss, acc, EERs[-1], min(EERs)))
		score_file.flush()

	if epoch >= args.max_epoch:
		quit()

	epoch += 1

100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:33<00:00, 46.52it/s]


2023-01-12 03:40:20 1 epoch, ACC 0.00%, EER 21.73%, bestEER 21.73%


100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:33<00:00, 46.68it/s]


2023-01-12 08:18:30 2 epoch, ACC 0.02%, EER 18.21%, bestEER 18.21%


100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:33<00:00, 46.70it/s]


2023-01-12 12:58:06 3 epoch, ACC 0.37%, EER 13.46%, bestEER 13.46%


100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:34<00:00, 46.00it/s]


2023-01-12 17:37:29 4 epoch, ACC 0.98%, EER 11.77%, bestEER 11.77%


In [ ]:
# Train subcenter_DAM128
while(1):
	## Training for one epoch
	loss, lr, acc = s.train_network(epoch = epoch, loader = trainLoader)

	## Evaluation every [test_step] epochs
	if epoch % args.test_step == 0:
		s.save_parameters(args.model_save_path + "/model_%04d.model"%epoch)
		EERs.append(s.eval_network(eval_list = args.eval_list, eval_path = args.eval_path)[0])
		print(time.strftime("%Y-%m-%d %H:%M:%S"), "%d epoch, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%"%(epoch, acc, EERs[-1], min(EERs)))
		score_file.write("%d epoch, LR %f, LOSS %f, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%\n"%(epoch, lr, loss, acc, EERs[-1], min(EERs)))
		score_file.flush()

	if epoch >= args.max_epoch:
		quit()

	epoch += 1

100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:36<00:00, 45.24it/s]


2023-01-13 11:18:50 3 epoch, ACC 1.60%, EER 12.54%, bestEER 12.54%


100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:35<00:00, 45.67it/s]


2023-01-13 13:05:49 4 epoch, ACC 4.80%, EER 10.54%, bestEER 10.54%


100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:35<00:00, 45.49it/s]


...

2023-01-14 22:27:53 23 epoch, ACC 37.77%, EER 2.87%, bestEER 2.81%


100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:39<00:00, 43.66it/s]


2023-01-15 00:15:41 24 epoch, ACC 38.59%, EER 2.68%, bestEER 2.68%


100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:41<00:00, 43.12it/s]


2023-01-15 02:01:18 25 epoch, ACC 39.38%, EER 2.83%, bestEER 2.68%


In [ ]:
# Train subcenter_DAM128
while(1):
	## Training for one epoch
	loss, lr, acc = s.train_network(epoch = epoch+26, loader = trainLoader)

	## Evaluation every [test_step] epochs
	if epoch % args.test_step == 0:
		s.save_parameters(args.model_save_path + "/model_%04d.model"%epoch)
		EERs.append(s.eval_network(eval_list = args.eval_list, eval_path = args.eval_path)[0])
		print(time.strftime("%Y-%m-%d %H:%M:%S"), "%d epoch, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%"%(epoch, acc, EERs[-1], min(EERs)))
		score_file.write("%d epoch, LR %f, LOSS %f, ACC %2.2f%%, EER %2.2f%%, bestEER %2.2f%%\n"%(epoch, lr, loss, acc, EERs[-1], min(EERs)))
		score_file.flush()

	if epoch >= args.max_epoch:
		quit()

	epoch += 1

100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:34<00:00, 45.94it/s]


2023-01-15 12:34:02 1 epoch, ACC 29.30%, EER 2.55%, bestEER 2.55%


100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:34<00:00, 46.24it/s]


2023-01-15 14:50:45 2 epoch, ACC 35.31%, EER 2.38%, bestEER 2.38%


100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:33<00:00, 46.49it/s]


...

2023-01-16 18:34:14 17 epoch, ACC 47.29%, EER 2.04%, bestEER 1.94%


100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:36<00:00, 44.99it/s]


2023-01-16 20:20:00 18 epoch, ACC 47.82%, EER 2.06%, bestEER 1.94%


100%|██████████████████████████████████████████████████████████████████████████████| 4356/4356 [01:37<00:00, 44.61it/s]


2023-01-16 22:03:36 19 epoch, ACC 48.30%, EER 1.94%, bestEER 1.94%


In [5]:
a = torch.randn((17, 5994, 192))

In [6]:
a.shape

torch.Size([17, 5994, 192])

In [7]:
a.mT.shape

torch.Size([17, 192, 5994])